In [1]:
# Cargamos el acceso a  nuestro drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# Nos movemos a la ruta y verificamos
%cd '/content/drive/My Drive/Colab Notebooks/db'
!ls

/content/drive/My Drive/Colab Notebooks/db
'Copia de london_merged.csv'   poblacion.csv   test.parquet   text.csv
 london_merged.csv	       test.hdf        test.pkl
 Meteorite_Landings.csv        test.json       test.xlsx


In [3]:
import pandas as pd
import numpy as np
pd.__version__

'1.0.5'

In [13]:
#Quitamos notacion cientifica
pd.options.display.float_format='{:,.1f}'.format

#Cargamos df
df_pob = pd.read_csv('poblacion.csv')
df_pob

,Country,year,pop
0,Afghanistan,2015,"34,413,603.0"
1,Albania,2015,"2,880,703.0"
2,Algeria,2015,"39,728,025.0"
3,American Samoa,2015,"55,812.0"
4,Andorra,2015,"78,011.0"
...,...,...,...
1035,Pre-demographic dividend,2018,"919,485,393.0"
1036,Small states,2018,"40,575,321.0"
1037,South Asia,2018,"1,814,388,744.0"
1038,South Asia (IDA & IBRD),2018,"1,814,388,744.0"


In [16]:
df_pob['year'] = pd.Categorical(df_pob['year'].apply(str))

In [17]:
df_pob.dtypes

Country      object
year       category
pop         float64
dtype: object

In [19]:
idx_filtro = df_pob['Country'].isin(['Aruba', 'Colombia'])
idx_filtro

0       False
1       False
2       False
3       False
4       False
        ...  
1035    False
1036    False
1037    False
1038    False
1039    False
Name: Country, Length: 1040, dtype: bool

In [22]:
df_sample = df_pob[idx_filtro]
df_sample

,Country,year,pop
9,Aruba,2015,"104,341.0"
42,Colombia,2015,"47,520,667.0"
269,Aruba,2016,"104,872.0"
302,Colombia,2016,"48,171,392.0"
529,Aruba,2017,"105,366.0"
562,Colombia,2017,"48,901,066.0"
789,Aruba,2018,"105,845.0"
822,Colombia,2018,"49,648,685.0"


In [23]:
# Creamos multiples indices
df_sample = df_sample.set_index(['Country', 'year']).sort_index()
df_sample

pop
Country  year             
Aruba    2015    104,341.0
         2016    104,872.0
         2017    105,366.0
         2018    105,845.0
Colombia 2015 47,520,667.0
         2016 48,171,392.0
         2017 48,901,066.0
         2018 49,648,685.0

In [24]:
# Obtenemos 1 pais
df_sample.loc['Colombia',:].loc['2016',:]

pop   48,171,392.0
Name: 2016, dtype: float64

In [28]:
# seleccionar multiples indices con la funcion xs()
df_sample.xs(['Aruba', '2018'])

pop   105,845.0
Name: (Aruba, 2018), dtype: float64

In [29]:
# xs por niveles
df_sample.xs('2018', level='year')

,pop
Country,
Aruba,"105,845.0"
Colombia,"49,648,685.0"


In [34]:
df_countries = df_pob.set_index(['Country', 'year']).sort_index(ascending=[True, True])
df_countries

pop
Country     year             
Afghanistan 2015 34,413,603.0
            2016 35,383,128.0
            2017 36,296,400.0
            2018 37,172,386.0
Albania     2015  2,880,703.0
...                       ...
Zambia      2018 17,351,822.0
Zimbabwe    2015 13,814,629.0
            2016 14,030,390.0
            2017 14,236,745.0
            2018 14,439,018.0

[1040 rows x 1 columns]

In [36]:
# Tomar pedazos del df con la funcion pd.IndexSlice
ids = pd.IndexSlice
df_countries.loc[ids['Aruba':'Austria', '2015':'2017'],:].sort_index()

pop
Country   year             
Aruba     2015    104,341.0
          2016    104,872.0
          2017    105,366.0
Australia 2015 23,815,995.0
          2016 24,190,907.0
          2017 24,601,860.0
Austria   2015  8,642,699.0
          2016  8,736,668.0
          2017  8,797,566.0

In [37]:
df_countries.index.get_level_values(0)

Index(['Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Albania',
       'Albania', 'Albania', 'Albania', 'Algeria', 'Algeria',
       ...
       'Yemen, Rep.', 'Yemen, Rep.', 'Zambia', 'Zambia', 'Zambia', 'Zambia',
       'Zimbabwe', 'Zimbabwe', 'Zimbabwe', 'Zimbabwe'],
      dtype='object', name='Country', length=1040)

In [38]:
df_countries.index.get_level_values(1)

CategoricalIndex(['2015', '2016', '2017', '2018', '2015', '2016', '2017',
                  '2018', '2015', '2016',
                  ...
                  '2017', '2018', '2015', '2016', '2017', '2018', '2015',
                  '2016', '2017', '2018'],
                 categories=['2015', '2016', '2017', '2018'], ordered=False, name='year', dtype='category', length=1040)

In [42]:
#Tomando datos de una columna tipo series usando cadenas de instrucciones
df_countries['pop']['Colombia']['2018']

49648685.0

In [44]:
# La gran ventaja de multiples indices es que podemos realizar operaciones matematicas sobre el
df_countries.sum(level='year')

,pop
year,
2015,"65,679,147,019.0"
2016,"66,487,930,677.0"
2017,"67,294,176,701.0"
2018,"68,087,886,692.0"


In [45]:
# Funcion un stack paraconvertir indices en columnas
df_countries.unstack('year')

pop                                       
year                          2015         2016         2017         2018
Country                                                                  
Afghanistan           34,413,603.0 35,383,128.0 36,296,400.0 37,172,386.0
Albania                2,880,703.0  2,876,101.0  2,873,457.0  2,866,376.0
Algeria               39,728,025.0 40,551,404.0 41,389,198.0 42,228,429.0
American Samoa            55,812.0     55,741.0     55,620.0     55,465.0
Andorra                   78,011.0     77,297.0     77,001.0     77,006.0
...                            ...          ...          ...          ...
Virgin Islands (U.S.)    107,710.0    107,510.0    107,268.0    106,977.0
West Bank and Gaza     4,270,092.0  4,367,088.0  4,454,805.0  4,569,087.0
Yemen, Rep.           26,497,889.0 27,168,210.0 27,834,821.0 28,498,687.0
Zambia                15,879,361.0 16,363,507.0 16,853,688.0 17,351,822.0
Zimbabwe              13,814,629.0 14,030,390.0 14,236,745.0 14,439,018.0

[260 rows x 4 columns]

In [46]:
df_countries.unstack('Country')

pop              ...                          
Country  Afghanistan     Albania  ...       Zambia     Zimbabwe
year                              ...                          
2015    34,413,603.0 2,880,703.0  ... 15,879,361.0 13,814,629.0
2016    35,383,128.0 2,876,101.0  ... 16,363,507.0 14,030,390.0
2017    36,296,400.0 2,873,457.0  ... 16,853,688.0 14,236,745.0
2018    37,172,386.0 2,866,376.0  ... 17,351,822.0 14,439,018.0

[4 rows x 260 columns]